# Project Basics Of Mobile Robotics

### Computer Vision


### Global navigation

A common pathfinding algorithm in computer science and artificial intelligence is the A* (A-star) algorithm. Its purpose is to determine the shortest path on a graph or grid between a starting point and a goal point. Combining the ideas of Greedy Best-First Search and Dijkstra's algorithm, A* uses a heuristic function to effectively direct the search.

Important attributes:

Optimality: If certain requirements are satisfied, A* ensures that the shortest path will be found. 

Heuristic Guidance: A* estimates the cost from the current node to the goal using a heuristic function, commonly referred to as the "h-value". By directing the search, this heuristic increases algorithmic efficiency by examining the most promising paths first.

Various methods can be employed to apply the A* algorithm on a map, including the fixed-grid size approach, Voronoi diagrams, visibility graphs, and cell decomposition methods. Each method offers distinct advantages and considerations for pathfinding applications.

In our project, we assessed two primary pathfinding approaches: the fixed-grid size method and the visibility graph. Although the visibility graph demonstrated a slightly faster computation time, the difference was not substantial. However, a critical drawback surfaced with the visibility graph method, as it significantly reduced the number of reachable points, resulting in suboptimal solutions.

You can see our visibility graph below: 

<img src="images\vis_graph.jpeg" alt="Visibility Graph" style="width: 500px;"/>


Given that computation time differences were marginal, we decided to favor the fixed-grid size approach. This method involves dividing the map into uniform squares of predetermined cell sizes and utilizing obstacle masks generated through computer vision to construct a grid. The A* algorithm is then applied to this grid to determine the most efficient path.

We are displaying the A* exploration in real time, and an screenshot of such display can be seen below:

<img src="images\Astar_explore.png" alt="Exploration" style="width: 500px;"/>

We are also displaying the grid and the found path on the the map. You can see the screenshot of them below:

<img src="images\Astar_path.png" alt="Path" style="width: 500px;"/>






In [ ]:
# imports
import heapq
import timeit
import cv2
import matplotlib.pyplot as plt
from computer_vision import * 
import numpy as np
import random

# possible moves from one square to another
moves_8n = [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]

moves_4n = [(0, -1), (0, 1), (-1, 0), (1, 0)]



class Node:
    """
    Represents a node in the A* algorithm.

    Attributes:
    - parent (Node): The parent node in the search tree.
    - position (Tuple[int, int]): The position (coordinates) of the node on the grid.
    - cost_of_move (float): The cost of reaching this node from the start.
    - heuristic (float): The estimated cost to reach the goal from this node.
    - total_cost (float): The sum of the cost_of_move and heuristic.

    Methods:
    - __eq__(self, other): Compares two nodes based on their positions.
    - __lt__(self, other): Compares two nodes based on their total_cost 
    
    """
    def __init__(self, parent=None, position=None):

        self.parent = parent
        self.position = position

        self.cost_of_move = 0

        self.heuristic = 0
        self.total_cost = 0


    def __eq__(self, other):

        return self.position == other.position


    def __lt__(self, other):

        if self.total_cost == other.total_cost:

            return id(self) < id(other)

        return self.total_cost < other.total_cost


def astar_grid(maze, start, end, moves, map_copy):
    """
    A* search algorithm for finding the shortest path on a 2D grid.

    Parameters:
    - maze (numpy.ndarray): A 2D grid where 0 represents an open path, and 1 represents an obstacle.
    - start (Tuple[int, int]): The starting point on the grid.
    - end (Tuple[int, int]): The destination point on the grid.
    - moves (List[Tuple[int, int]]): A list of tuples representing possible moves from a given position.

    Returns:
    - np.array[Tuple[int, int]] or None: The shortest path from the start to the end on the grid, or None if no path is found.

    Node Class:
    - The algorithm uses a Node class to represent a point in the search space, which includes information about the node's position,
      cost of movement from the start, heuristic (estimated cost to reach the goal), and the total cost.

    Assumptions:
    - The maze is represented as a NumPy array where 0 denotes an open path, and 1 denotes an obstacle.

    Example:
    >>> maze = np.array([[0, 0, 0, 1, 0],
    ...                  [0, 1, 0, 1, 0],
    ...                  [0, 1, 0, 0, 0],
    ...                  [0, 0, 0, 1, 0],
    ...                  [0, 0, 0, 0, 0]])
    >>> start = (0, 0)
    >>> end = (4, 4)
    >>> moves_8n = [(1, 0), (-1, 0), (0, 1), (0, -1), (1, 1), (-1, -1), (-1, 1), (1, -1)]
    >>> path = astar_grid(maze, start, end, moves_8n)
    """

    start_node = Node(None, start)

    start_node.cost_of_move = start_node.heuristic = start_node.total_cost = 0

    end_node = Node(None, end)

    end_node.cost_of_move = end_node.heuristic = end_node.total_cost = 0

    open_list = []
    closed_set = set()
    visited_positions = set()  # Maintain a set of visited positions

    heapq.heappush(open_list, start_node)
    visited_positions.add(start)

    while open_list:

        current_node = heapq.heappop(open_list)

        closed_set.add(current_node.position)

        cv2.circle(map_copy, (int((current_node.position[0] * map_copy.shape[1]) / len(maze[0])),
                              int((current_node.position[1] * map_copy.shape[0]) / len(maze))), 2, (125, 0, 55), -1)
        cv2.imshow('Map_copy', map_copy)
        cv2.waitKey(1)

        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)

                current_node = current_node.parent

            return path[::-1]

        children = [
            Node(current_node, (current_node.position[0] + dx, current_node.position[1] + dy))

            for dx, dy in moves
            if (

                    0 <= current_node.position[0] + dx < len(maze[0])

                    and 0 <= current_node.position[1] + dy < len(maze)

                    and maze[current_node.position[1] + dy][current_node.position[0] + dx] == 0

                    and (current_node.position[0] + dx, current_node.position[1] + dy) not in closed_set
                    and (current_node.position[0] + dx, current_node.position[1] + dy) not in visited_positions
            )
        ]

        for child in children:

            dx, dy = child.position[0] - current_node.position[0], child.position[1] - current_node.position[1]

            child.cost_of_move = current_node.cost_of_move + 1.41 if dx != 0 and dy != 0 else current_node.cost_of_move + 1

            child.heuristic = np.sqrt(abs(child.position[0] - end_node.position[0])**2 \
                              + abs(child.position[1] - end_node.position[1])**2)

            child.total_cost = child.cost_of_move + child.heuristic

            heapq.heappush(open_list, child)
            visited_positions.add(child.position)



def simplify_path(path):

    """
    Simplifies a path by removing unnecessary intermediate points.

    Parameters:
    - path (np.array[Tuple[int, int]]): The original path represented as a list of coordinates.

    Returns:
    - np.array[Tuple[int, int]]: The simplified path with unnecessary intermediate points removed.
    
    Example:
    >>> simplify_path([(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2)])
    [(0, 0), (3, 0), (3, 2)]

    """
    simplified_path = [path[0]] 

    for i in range(1, len(path) - 1):
        current_point = np.array(path[i])
        next_point = np.array(path[i + 1])
        direction_vector = next_point - np.array(simplified_path[-1])
        
        if np.cross(direction_vector, current_point - np.array(simplified_path[-1])) == 0:
            continue  

        simplified_path.append(path[i]) 

    simplified_path.append(path[-1])

    return simplified_path

def make_path(map_img, obstacle_masks, cell_size, start, end, grid,  metric_padding, map_x = 600, map_y = 400):
    """
    Generates a path on a grid-based map using A* algorithm, considering obstacles on the image.

    Parameters:
    - map_img (numpy.ndarray): The original map image.
    - obstacle_masks (List[numpy.ndarray]): List of obstacle masks on the map.
    - cell_size (int): Size of each grid cell.
    - start (Tuple[int, int]): Starting point in image coordinates (x, y).
    - end (Tuple[int, int]): Ending point in image coordinates (x, y).
    - grid (numpy.ndarray): The grid representing the map.
    - map_x (int): Width of the map in image coordinates.
    - map_y (int): Height of the map in image coordinates.

    Returns:
    - Tuple[numpy.ndarray, List[Tuple[int, int]], List[Tuple[int, int]], List[Tuple[float, float]]]:
        - grid (numpy.ndarray): The grid with obstacle information.
        - path_grid (List[Tuple[int, int]]): The path on the grid coordinates.
        - simplified_path (List[Tuple[int, int]]): The simplified path on the grid coordinates.
        - metric_path (List[Tuple[float, float]]): The path in metric (image) coordinates.

    Example:
    >>> map_img = cv2.imread('map_image.png')
    >>> obstacle_masks = [obstacle_mask_1, obstacle_mask_2]
    >>> cell_size = 10
    >>> start = (100, 50)
    >>> end = (300, 200)
    >>> grid, path_grid, simplified_path, metric_path = make_path(map_img, obstacle_masks, cell_size, start, end, grid)
    """
    map_copy = map_img.copy()
    bw_map = cv2.cvtColor(map_img.copy(), cv2.COLOR_BGR2GRAY)
    grid = create_grid(bw_map, obstacle_masks, cell_size, metric_padding)
    start_grid = (grid.shape[1] * start[0] // map_img.shape[1], grid.shape[0] * start[1] // map_img.shape[0])
    end_grid = (grid.shape[1] * end[0] // map_img.shape[1], grid.shape[0] * end[1] // map_img.shape[0])
    path_grid = astar_grid(grid, start_grid, end_grid, moves_8n, map_copy)
    
    if path_grid is None:
        print("no path found")
        return grid, None, None, None
    simplified_path = simplify_path(path_grid)
    metric_path = transform_grid_to_metric(simplified_path, map_x, map_y, grid)

    return grid, path_grid, simplified_path, metric_path, map_copy 


def transform_grid_to_metric(path, map_width, map_height, grid):
    """
    Transforms a path from grid coordinates to metric (image) coordinates.

    Parameters:
    - path (List[Tuple[int, int]]): The path in grid coordinates.
    - map_width (int): Width of the map in metric (image) coordinates.
    - map_height (int): Height of the map in metric (image) coordinates.
    - grid (numpy.ndarray): The grid representing the map.

    Returns:
    - List[Tuple[float, float]]: The path in metric (image) coordinates.

    Example:
    >>> path = [(0, 0), (3, 0), (3, 2)]
    >>> map_width = 600
    >>> map_height = 400
    >>> grid = np.zeros((4, 4), dtype=int)
    >>> metric_path = transform_grid_to_metric(path, map_width, map_height, grid)
    """
    metric_path = []
    grid_x = grid.shape[1]
    grid_y = grid.shape[0]
    for x,y in path:
        metric_path.append(((x * map_width) / grid_x, (y * map_height) / grid_y))

    return metric_path


if __name__ == "__main__":
    pass

# Bayesian Filter
This section has been written and the code made by Jean Cordonnier
## Introduction
We decided to go with a kalman filter as our bayesian filter. Our choice has been made based on 2 factors: it does not take big computational power (like the particle filter) and we assumed gaussian noise on the system. To implement the filter we used a python library from GitHub : https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/. The installation is in requirement.txt. 

To deal with the non-linearities in the orientation and position (we needed to change frame using the angle of the robot for the acceleration ad the velocity) and to deal with the format of the measurement (position ,velocity and acceleration being 2 dimensions variables and heading and spin (dheading/dt) being one dimensionnal) we decided to go with two kalmans: One for the position and one for the orientation. We will refer to them as kf_pos and kf_rot. Both are implemented in a kalman class defined in kalman.py
## Kalman for Position implementation
### Choice for measurement and state variables
For our measurements we used the data from the accelerometers, the average of the velocity of the wheels and the position from the camera. 
The kalman coordinates are exprimed in the local frame (with 0,0 being the top left corner) and the velocity and the accelerometers were exprimed in the body frame (centered with the robot) we had to do a coordinate change represented by this function: 
```python
#Change from body frame to local frame
def change_frame(body_data,body_orientation):
    '''
    Change from body frame to local frame.

    Parameters:
    - body_data: Data expressed in the body frame.
    - body_orientation: Orientation of the robot in the local frame (from kf_rot).

    Returns:
    Data expressed in the local frame.
    '''
    R = np.array([[np.cos(body_orientation), -np.sin(body_orientation)],
                  [np.sin(body_orientation), np.cos(body_orientation)]])
    accel_local = R.dot(body_data)
    return accel_local
```
The kf_pos has the following state variables (in the local frame): [positionx, positiony, velocityx, velocityy, accelerationx, accelerationy]. We also needed to define the F matrix linking the previous state to the next state such that x_next = F*x_previous. We defined it based on simple movement equation. Since the measurements dont have the same refresh rate we have to update the F matrix at every iteration depending on dt. 
```python
self.kf_pos.F = np.array([[1,0,dt,0,0.5*dt**2,0],
                          [0,1,0,dt,0,0.5*dt**2],
                          [0,0,1,0,dt,0],
                          [0,0,0,1,0,dt],
                          [0,0,0,0,1,0],
                          [0,0,0,0,0,1]])
```
One of the challenge we had to face was to manage to give the measurements with different frequencies. The solution we chose was to declare only two measurement in the declaration of the function and adapt the H matrix (linking the measurement z and the state x such that z = H*x) depending on which type of measurement it was. Which gave those results: 
```python
#for the acceleration
self.kf_pos.H = np.array([[0,0,0,0,1,0],
                         [0,0,0,0,0,1]])
                         #for the acceleration
self.kf_pos.H = np.array([[0,0,1,0,0,0],
                         [0,0,0,1,0,0]])
#for the acceleration
self.kf_pos.H = np.array([[1,0,0,0,0,0],
                         [0,1,0,0,0,0]])
```

The other challenge was to compute the process noise matrix Q. We assumed the noises to be independent so the non-diagonal terms had to be zero. For the diagonal terms we found our answer here: https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python/blob/master/07-Kalman-Filter-Math.ipynb. We don't really understand the 'why' of the math behind it but the kalman converged with this Q matrix so we are happy with it (and it did not work with other tries)
The final implementation of the Q matrix was this: 
```python
#compute Q according to dt 
        self.kf_pos.Q = np.array([
        [1/4 * noise**2 * dt**4, 0, 0, 0, 0, 0],
        [0, 1/4 * noise**2 * dt**4, 0, 0, 0, 0],
        [0, 0, noise**2 * dt**2, 0, 0, 0],
        [0, 0, 0, noise**2 * dt**2, 0, 0],
        [0, 0, 0, 0, noise**2, 0],
        [0, 0, 0, 0, 0, noise**2]])
```
In the end we had to compute the noise of each measurement. We assumed it to be white noise for all measurement and to be equivalent for the x and y part. The process to calculate the noise will be detailled below. For the implementation of the measurement noise matrix we used diagonal matrix with the covariance of the measurement: 

```python
#for acceleration
self.kf_pos.R = (self.ACCEL_NOISE**2)*np.eye(2)
#for velocity
self.kf_pos.R = (self.VEL_NOISE**2)*np.eye(2)
#for position
self.kf_pos.R = (self.POS_NOISE**2)*np.eye(2)
```
We also had to initalize our start variable and the trust we have in it and then we were good to go! Our kalman loop looks like this (here for the acceleration measurement only but the idea doesnt change for others measurements)
```python 
#call this line when you want to update the accelerometer measurement
kf_pos.update_acceleration([accx,accy],current_time)

def update_acceleration(self,data,time):
    
        # calculate time since last measurement and update it
        dt = time- self.time_pos
        self.time_pos = time

        #get current rotation from other kalman
        rotation = self.kf_rot.x.T[0]

        # Transform body frame to local frame
        data = change_frame(data, rotation)

        # Save measurement for plotting
        self.accel_measurement.append(data)

        # Update kf parameters to accept accel measurement
        self.kf_pos.H = np.array([[0,0,0,0,1,0],
                                  [0,0,0,0,0,1]])
        self.kf_pos.R = (self.ACCEL_NOISE**2)*np.eye(2)

        #Update kalman
        self._compute_kf_pos(data,dt)


# Same for velocity, position and acceleration measurement
# Update the F matrix, Q matrix and update the measurement for the position kalman
def _compute_kf_pos(self,data, dt):

    # get accel_noise for process noise matrix
    noise = self.ACCEL_NOISE # most dominant noise

    # compute F according to dt
    self.kf_pos.F = np.array([[1,0,dt,0,0.5*dt**2,0],
                    [0,1,0,dt,0,0.5*dt**2],
                    [0,0,1,0,dt,0],
                    [0,0,0,1,0,dt],
                    [0,0,0,0,1,0],
                    [0,0,0,0,0,1]])
    
    # compute Q according to dt 
    self.kf_pos.Q = np.array([
    [1/4 * noise**2 * dt**4, 0, 0, 0, 0, 0],
    [0, 1/4 * noise**2 * dt**4, 0, 0, 0, 0],
    [0, 0, noise**2 * dt**2, 0, 0, 0],
    [0, 0, 0, noise**2 * dt**2, 0, 0],
    [0, 0, 0, 0, noise**2, 0],
    [0, 0, 0, 0, 0, noise**2]])

    #predict and update kalman
    self.kf_pos.predict()
    self.kf_pos.update(data)
```
## Kalman for rotation 

The kalman for rotation is very similar to the kf_pos except that the measurements differ ( and there is not any frame change). We used as state variable (and measurement also) the heading (from the camera) and the heading rate change (from the wheel speed difference). As the implementation is easier than kf_pos we will not review his full implementation as we can just adapt the steps presented above. 

## Noise processing and data acquisition

For the noise calculation and the measurement equivalence we did the following: 
Position x and position y : Measurement: from the camera. Noise: make the robot stand still with the camera looking at it. Save the camera measurements of the position and extract the covariance. 
Velocity x and Velocity y : Measurement: fix identique speed for both wheel. Measure the time to cover a known distance and deduce a linear speed for all motor input. Always output the average of the two wheels and make it go through the rotation matrix. Noise: We fine tuned it using the position from the camera as a ground truth
Acceleration x and acceleration y: Measurement: from the accelerometer, we changed the value taking the gravity as a reference. Npise: as the accelerometer were giving stable values when the thymio was stopped we use the 9.81/21(equivalent of gravity for accelerometer) as a value. We increased it by 10 later on since it seemed very inaccurate (accelerometers were not precise enough)
Heading change (spin): Measurement: from the wheel. We gave an opposite speed to the 2 wheels than we calculated the time to make 10 turn around itself and calculated the spin in rad/s. From this we computed a linear relation between the spin and the wheel speed difference. Noise: We fined tuned it using the heading from the camera as a ground truth.
Heading: Measurement: from the camera. Noise: make the robot stand still with the camera looking at it. Save the camera measurements of the heading and extract the covariance.

In the end our noises were declared in the kalman class with those values:

``` python
 ACCEL_NOISE = 10*9.81/23
VEL_NOISE = 0.001
ROT_NOISE = 0.005
POS_NOISE = 0.005
SPIN_NOISE = 0.1
```


## Simulation of the kalman

All the simulation of the kalman have been made in kalman.ipynb. We created artificial sensor data and we added white noise to it. We also tested the class we create in test_kalman_class.ipynb (but note that since the noise value are not accurate the result don't fit the curve perfectly but it still is a good example on how to use the Kalman class with rotation and acceleration data). In kalman.ipynb we simulated acceleration measurement and position measurement then stopped position measurement to only rely on accelerometer data. All measurements are in local frame.


<img src="images/kalman_simulation_position.png" alt="Results for the position, the initial position starts at (25,0)" width="400"/>

Legend: Results for the position, the initial position starts at (25,0). We can see that despite having no GPS data the position estimation fits really well the ground truth but drift slowly away from it. The estimation converges quite quickly to the ground truth position in the beginning

<img src="images/kalman_simulation_speed.png" alt="Results for the speed" width="400"/>

Legend: Results for the speed. We can see that it matches the ground truth but it takes somes times to converge when the speed changes. 

<img src="images/kalman_simulation_acceleration.png" alt="Results for the acceleration" width="400"/>

Legend: Results for the acceleration. We can see that we start hitting the limit of the kalman since the shift becomes more important. The results are still accurate


## Real application of the kalman

Now it was time to test our kalman in the robot. We made the robot follow a path, hid the camera for a bit and saw what is gave us. It is important to notice that make the robot working with bluetooth and the sensor values were a bit shifted over time. 

<img src="images/Kalman_setup.png" alt="Kalman results" width="400"/>

Legend: Picture of the kalman path

<img src="images/Kalman_position.png" alt="Kalman results" width="1100"/>

Legend: Kalman position vs Camera measurement. The robot starts on the bottom left. We can see that the kalman estimates the path perfectly when there is camera measurement. When there is no camera we can see that the inertial navigation does the job even though it drifts a little over time. The results tend to differ from one try to another (espcially when it is connected via cable or bluetooth). In general, the inertial navigation allows the thymio to go across the whole board (long side) with 2-3 turn with a +- good approximation on the position. 

<img src="images/Kalman_graph.png" alt="Kalman results" width="1100"/>

Legend: Kalman state variable vs Measured one. In general we can see that we have a good fit between the data measurement and estimation. We have a big shift between the kalman and the camera heading. We think part of it is due to the bluetooth connection as it diseappars whith a wired connection. Also the initial position are not saved so there is one point missing for the kalman wich can explain part of the shift. We can see that we can only rely on inertial navigation the kalman data tends to match more the sensors data.


## Kalman Conclusion

Globally we are happy with the kalman results as it allows us to perform robust localisation and inertial navigation to a certain extend. We think that using an extended KF could have helped us get better results that converges more easily. We would say that the filtering part is successful





##### Start the client and wait for a lock

In [ ]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

### Local navigation